In [1]:
%%time

import boto3
import findspark
import os
import pandas as pd

findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.pandas as ps

from IPython.core.display import HTML

display(HTML("<style>pre { white-space: pre !important; }</style>"))

os.environ['PYARROW_IGNORE_TIMEZONE'] = str(1)

spark = SparkSession.builder \
    .appName("testing") \
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.access.key", os.environ['AWS_ACCESS_KEY_ID']) \
    .config("spark.hadoop.fs.s3a.secret.key", os.environ['AWS_SECRET_ACCESS_KEY']) \
    .config("spark.hadoop.fs.s3a.proxy.host", "s3") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://s3:9000") \
    .config("spark.hadoop.fs.s3a.port", "9000") \
    .getOrCreate()

print("Spark inicializado com sucesso com as credenciais do S3")

Spark inicializado com sucesso com as credenciais do S3
CPU times: user 1.43 s, sys: 529 ms, total: 1.95 s
Wall time: 19.7 s


In [2]:
spark

In [ ]:
spark.read.csv("output.csv")